#### Write a Python notebook that creates a vector database using ChromaDB (use LangChain)
- ingest the document files only (full_ItemID.html files)
- it is required to save the file path in the metadata

In [1]:
import os
from langchain.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
from readability.readability import Document

d:\PhapDien_semantic_search\phapdienvv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [50]:
# Step 1: Define the directory containing the HTML files
input_dir = rf"D:\PhapDien_semantic_search\BoPhapDienDienTu\vbpl"
model = SentenceTransformer('bkai-foundation-models/vietnamese-bi-encoder')

# Step 2: Read and clean the HTML files
def load_and_clean_html(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        html_content = f.read()
    soup = BeautifulSoup(html_content, "html.parser")
    text = soup.get_text()  # Extract plain text from the HTML
    return text

# Step 3: Process all files in the directory
documents = []
metadata = []
for file_name in os.listdir(input_dir)[:5]:
    if file_name.startswith("full_") and file_name.endswith(".html"):
        file_path = os.path.join(input_dir, file_name)
        text = load_and_clean_html(file_path)
        documents.append(text)
        metadata.append({"file_path": file_path})

print(f"Loaded {len(documents)} documents")
# Step 4: Split text into chunks
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base", chunk_size=2000, chunk_overlap=20, separator="\n"
)

splitted_docs = []
splitted_metadata = []

for doc, meta in zip(documents, metadata):
    chunks = text_splitter.split_text(doc)
    for chunk in chunks:
        splitted_docs.append(chunk)
        splitted_metadata.append(meta)
# Step 5: Naive text cleaning: for each chunk, remove extra whitespaces and newlines, remove text components less than 50 characters.
# keep information
processed_splitted_docs = []
for doc in splitted_docs:
    processed = doc.split("\n")
    for phrase in processed:
        if len(phrase) > 50:
            processed_splitted_docs.append(phrase)
    # processed_splitted_docs.extend(processed)

print(processed_splitted_docs)   

Loaded 5 documents
[' Địa phương                            Danh sách các tỉnh & thành phố trực thuộc Trung ương                                        Thành phố                                                                     Các tỉnh                                     Địa Phương Hà Nội TP HCM Đà Nẵng Hải Phòng Cần Thơ     An Giang Bà Rịa - Vũng Tàu Bắc Giang Bắc Kạn Bạc Liêu Bắc Ninh Bến Tre Bình Định Bình Dương Bình Phước Bình Thuận Cà Mau Cao Bằng Đắk Lắk Đắk Nông     Điện Biên Đồng Nai Đồng Tháp Gia Lai Hà Giang Hà Nam Hà Tĩnh Hải Dương Hậu Giang Hòa Bình Hưng Yên Khánh Hòa Kiên Giang Kon Tum Lai Châu     Lâm Đồng Lạng Sơn Lào Cai Long An Nam Định Nghệ An Ninh Bình Ninh Thuận Phú Thọ Phú Yên Quảng Bình Quảng Nam Quảng Ngãi Quảng Ninh Quảng Trị     Sóc Trăng Sơn La Tây Ninh Thái Bình Thái Nguyên Thanh Hóa Thừa Thiên Huế Tiền Giang Trà Vinh Tuyên Quang Vĩnh Long Vĩnh Phúc Yên Bái        ', ' Các Bộ, Ngành                                            Danh sách các Bộ, Ngành        

In [ ]:
# Step 6: Generate embeddings using BKAI model
# embeddings = model.encode(sentences)
# print(f"Embedding shape: {embeddings.shape}")

# # Step 7: Save the vectors to ChromaDB
# vector_db = Chroma.from_texts(
#     texts=splitted_docs,
#     embedding=embeddings,
#     metadatas=splitted_metadata,
#     persist_directory="chroma_db"  # Directory where the database will be saved
# )

# # Step 8: Persist the database to disk
# vector_db.persist()
# print("Database saved successfully!")
